In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/optiver_real_vol')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/bin/bash: line 1: nvidia-smi: command not found
Your runtime has 13.6 gigabytes of available RAM

Not using a high-RAM runtime


In [ ]:

import pandas as pd
import numpy as np

import glob
import pickle

import matplotlib.pyplot as plt

from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.linear_model import LinearRegression

import scipy.stats as stats

### DATA MUNGING for HAR features extraction

In [ ]:
# """
# HAR models forecasting realized volatility

# """


# def log_return(list_stock_prices):
#     return np.log(list_stock_prices).diff()

# def realized_volatility(series_log_return):
#     return np.sqrt(np.sum(series_log_return**2))

# def realized_quarticity(log_returns):
#     return np.sum(log_returns**4) / len(log_returns)

# def bipower_variation(log_returns):
#     log_returns = log_returns.values
#     bpv = 0
#     for i in range(len(log_returns) - 1):
#         bpv += np.abs(log_returns[i] * log_returns[i+1])
#     return (np.sqrt(2 / np.pi)) ** (-2) * bpv

# def positive_realized_semivariance(log_returns):
#     log_returns = log_returns.values
#     positive_returns = log_returns[log_returns > 0]
#     return np.sum(positive_returns**2) / len(positive_returns)

# def negative_realized_semivariance(log_returns):
#     log_returns = log_returns.values
#     negative_returns = log_returns[log_returns < 0]
#     return np.sum(negative_returns**2) / len(negative_returns)


In [ ]:
# def book_wap1_HAR_estimator(subset_paths,train_target):

#     book_wap1_HAR_df = pd.DataFrame()

#     for path in subset_paths:
#         print(path)
#         st_id = int(path.split('/')[1].split('_')[1].split('=')[1])
#         # stock ids in [103,18,31,37] have DIFFERENT length of time_id compared to target so we exclude them for now
#         if st_id != -1:#not in [103,18,31,37,110]: # select stock id here
#           df_book_data = pd.read_parquet(path)
#           df_book_data['wap'] =(df_book_data['bid_price1'] * df_book_data['ask_size1']+df_book_data['ask_price1'] * df_book_data['bid_size1'])  / (
#                                       df_book_data['bid_size1']+ df_book_data[
#                                   'ask_size1'])
#           df_book_data['log_ret_price'] = df_book_data.groupby(['time_id'])['wap'].apply(log_return).values
#           df_book_data = df_book_data[~df_book_data['log_ret_price'].isnull()]
#           train_st = df_book_data
#           df_realized_vol_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(realized_volatility)).rename(columns={'log_ret_price':'book_real_vol_price'})
#           df_realized_quart_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(realized_quarticity)).rename(columns={'log_ret_price':'book_real_quart_price'})
#           df_realized_bpv_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(bipower_variation)).rename(columns={'log_ret_price':'book_real_bpv_price'})
#           df_realized_pos_semvar_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(positive_realized_semivariance)).rename(columns={'log_ret_price':'book_real_pos_semvar_price'})
#           df_realized_neg_semvar_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(negative_realized_semivariance)).rename(columns={'log_ret_price':'book_real_neg_semvar_price'})

#           target_st =train_target[train_target['stock_id']==st_id] # use when not called above
#           target_st.index = [target_st["time_id"]] # use when not called above
#           common_time_id = np.intersect1d(df_realized_vol_per_stock.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
#           har_estimates_df = pd.DataFrame()
#           har_estimates_df['book_real_vol_price'] = df_realized_vol_per_stock.loc[common_time_id,"book_real_vol_price"].values
#           har_estimates_df['book_real_quart_price'] = df_realized_quart_per_stock.loc[common_time_id,"book_real_quart_price"].values
#           har_estimates_df['book_real_bpv_price'] = df_realized_bpv_per_stock.loc[common_time_id,"book_real_bpv_price"].values
#           har_estimates_df['book_real_pos_semvar_price'] = df_realized_pos_semvar_per_stock.loc[common_time_id,"book_real_pos_semvar_price"].values
#           har_estimates_df['book_real_neg_semvar_price'] = df_realized_neg_semvar_per_stock.loc[common_time_id,"book_real_neg_semvar_price"].values
#           har_estimates_df['target_vol'] = target_st.loc[common_time_id,"target"].values
#           har_estimates_df['st_id'] = np.repeat(st_id,len(common_time_id))
#           har_estimates_df['time_id'] = common_time_id

#           book_wap1_HAR_df = pd.concat([book_wap1_HAR_df,har_estimates_df])

#     return book_wap1_HAR_df

In [ ]:

# def trade_HAR_estimator(subset_paths,train_target):

#     trade_HAR_df = pd.DataFrame()

#     for path in subset_paths:
#         print(path)
#         st_id = int(path.split('/')[1].split('_')[1].split('=')[1])
#         # stock ids in [103,18,31,37] have DIFFERENT length of time_id compared to target so we exclude them for now
#         if st_id != -1:#not in [103,18,31,37,110]: # select stock id here
#           train_st = pd.read_parquet(path)
#           ## compute realized volality in the first 10 mins using trade execution price (instead of WAP)
#           train_st['log_ret_price'] = train_st.groupby(by='time_id')['price'].apply(log_return).values
#           train_st = train_st[~train_st['log_ret_price'].isnull()]
#           df_realized_vol_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(realized_volatility)).rename(columns={'log_ret_price':'real_vol_price'})
#           df_realized_quart_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(realized_quarticity)).rename(columns={'log_ret_price':'real_quart_price'})
#           df_realized_bpv_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(bipower_variation)).rename(columns={'log_ret_price':'real_bpv_price'})
#           df_realized_pos_semvar_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(positive_realized_semivariance)).rename(columns={'log_ret_price':'real_pos_semvar_price'})
#           df_realized_neg_semvar_per_stock =  pd.DataFrame(train_st.groupby(['time_id'])['log_ret_price'].agg(negative_realized_semivariance)).rename(columns={'log_ret_price':'real_neg_semvar_price'})

#           target_st =train_target[train_target['stock_id']==st_id] # use when not called above
#           target_st.index = [target_st["time_id"]] # use when not called above
#           common_time_id = np.intersect1d(df_realized_vol_per_stock.index.values, target_st['time_id'].values) # ensure that only common time ids in trade_train and target are used.
#           har_estimates_df = pd.DataFrame()
#           har_estimates_df['real_vol_price'] = df_realized_vol_per_stock.loc[common_time_id,"real_vol_price"].values
#           har_estimates_df['real_quart_price'] = df_realized_quart_per_stock.loc[common_time_id,"real_quart_price"].values
#           har_estimates_df['real_bpv_price'] = df_realized_bpv_per_stock.loc[common_time_id,"real_bpv_price"].values
#           har_estimates_df['real_pos_semvar_price'] = df_realized_pos_semvar_per_stock.loc[common_time_id,"real_pos_semvar_price"].values
#           har_estimates_df['real_neg_semvar_price'] = df_realized_neg_semvar_per_stock.loc[common_time_id,"real_neg_semvar_price"].values
#           har_estimates_df['target_vol'] = target_st.loc[common_time_id,"target"].values
#           har_estimates_df['st_id'] = np.repeat(st_id,len(common_time_id))
#           har_estimates_df['time_id'] = common_time_id

#           trade_HAR_df = pd.concat([trade_HAR_df,har_estimates_df])

#     return trade_HAR_df

In [ ]:
"""
Create HAR features for trade data
"""

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/optimus_GPU_data/data')
# subset_paths = glob.glob('trade_train.parquet/stock_id=*')
# train_target = pd.read_csv('train.csv')
# trade_HAR_vol_estimates_df = trade_HAR_estimator(subset_paths,train_target)
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# pickle.dump(trade_HAR_vol_estimates_df, open('trade_HAR_vol_estimates_df.pkl', 'wb'))


'\nCreate HAR features for trade data\n'

In [ ]:
"""
Create HAR features for book data
"""

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/optimus_GPU_data/data')
# subset_paths = glob.glob('book_train.parquet/stock_id=*')
# train_target = pd.read_csv('train.csv')
# book_wa1_HAR_vol_estimates_df = book_wap1_HAR_estimator(subset_paths,train_target)
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# pickle.dump(book_wa1_HAR_vol_estimates_df, open('book_wa1_HAR_vol_estimates_df.pkl', 'wb'))


'\nCreate HAR features for book data\n'

In [ ]:
"""
reordering time_ids to correct order for trade wap1 data
"""

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
# correct_time_id_order = pickle.load(open('correct_time_id_order.pkl','rb'))

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# trade_HAR_vol_estimates_df = pickle.load(open('trade_HAR_vol_estimates_df.pkl', 'rb'))


# # reorder time_ids to correct order

# def my_reorder_stock_in_df(df, time_ids_reordered):
#     common_values = [value for value in time_ids_reordered if value in df['time_id'].values]
#     df = df.set_index('time_id')
#     df = df.reindex(common_values)
#     df['time_id'] = df.index
#     return df

# # Assuming you have a dataframe called 'df' and an array of reordered time_ids called 'time_ids_reordered'
# HAR_trade_data_reordered = trade_HAR_vol_estimates_df.groupby('st_id').apply(my_reorder_stock_in_df, time_ids_reordered=correct_time_id_order).reset_index(drop=True)
# HAR_trade_data_reordered


# #### Load the dataset
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# HAR_book_wap1_data_reordered = pickle.load(open('book_wap1_HAR_vol_estimates_reordered_df.pkl', 'rb'))
# temp_df = HAR_book_wap1_data_reordered.drop(columns=['target_vol', 'book_real_vol_price', 'book_real_quart_price', 'book_real_bpv_price', 'book_real_pos_semvar_price', 'book_real_neg_semvar_price'])
# temp_df

# ### fill the missing time ids in trade data to match with length of book data
# HAR_trade_data_reordered_df = pd.merge(temp_df,HAR_trade_data_reordered, left_on=['st_id','time_id'], right_on=['st_id','time_id'], how='left').ffill().bfill()
# HAR_trade_data_reordered_df

# #### save the dataset
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# pickle.dump(HAR_trade_data_reordered_df,open('trade_HAR_vol_estimates_reordered_df.pkl', 'wb'))

'\nreordering time_ids to correct order for trade wap1 data\n'

In [ ]:
"""
reordering time_ids to correct order for book data
"""

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets')
# correct_time_id_order = pickle.load(open('correct_time_id_order.pkl','rb'))

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# book_wap1_HAR_vol_estimates_full_df = pickle.load(open('book_wap1_HAR_vol_estimates_full_df.pkl', 'rb'))

# # reorder time_ids to correct order

# def my_reorder_stock_in_df(df, time_ids_reordered):
#     common_values = [value for value in time_ids_reordered if value in df['time_id'].values]
#     df = df.set_index('time_id')
#     df = df.reindex(common_values)
#     df['time_id'] = df.index
#     return df

# # Assuming you have a dataframe called 'df' and an array of reordered time_ids called 'time_ids_reordered'
# HAR_book_wap1_data_reordered = book_wap1_HAR_vol_estimates_full_df.groupby('st_id').apply(my_reorder_stock_in_df, time_ids_reordered=correct_time_id_order).reset_index(drop=True)
# HAR_book_wap1_data_reordered

# #### Load the dataset
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# pickle.dump(HAR_book_wap1_data_reordered,open('book_wap1_HAR_vol_estimates_reordered_df.pkl', 'wb'))



'\nreordering time_ids to correct order for book data\n'

#### Training the HAR model for prediction

In [ ]:
# """
# load book wap1 HAR features
# """
# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# book_wap1_HAR_vol_estimates_full_df = pickle.load(open('book_wap1_HAR_vol_estimates_reordered_df.pkl', 'rb'))

# book_wap1_HAR_vol_estimates_full_df = book_wap1_HAR_vol_estimates_full_df.rename(columns={'book_real_vol_price':'RV','book_real_quart_price': 'RQ', 'book_real_bpv_price': 'BPV', 'book_real_pos_semvar_price': 'RV_plus', 'book_real_neg_semvar_price': 'RV_minus'})
# book_wa1_HAR_vol_estimates_df = book_wap1_HAR_vol_estimates_full_df.drop(columns=['target_vol','st_id',	'time_id'])
# # book_wa1_HAR_vol_estimates_df



In [ ]:
# """
# load trade HAR features
# """

# os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
# trade_HAR_vol_estimates_full_df = pickle.load(open('trade_HAR_vol_estimates_reordered_df.pkl', 'rb'))

# trade_HAR_vol_estimates_full_df = trade_HAR_vol_estimates_full_df.rename(columns={'real_vol_price':'RV','real_quart_price': 'RQ', 'real_bpv_price': 'BPV', 'real_pos_semvar_price': 'RV_plus', 'real_neg_semvar_price': 'RV_minus'})
# trade_HAR_vol_estimates_df = trade_HAR_vol_estimates_full_df.drop(columns=['target_vol','st_id',	'time_id'])

# ## reorder columns
# st_id = trade_HAR_vol_estimates_full_df['st_id']
# time_id =  trade_HAR_vol_estimates_full_df['time_id']
# trade_HAR_vol_estimates_full_df = trade_HAR_vol_estimates_full_df.drop(columns=['st_id',	'time_id'])
# trade_HAR_vol_estimates_full_df['st_id'] = st_id
# trade_HAR_vol_estimates_full_df['time_id'] = time_id
# trade_HAR_vol_estimates_full_df

# #trade_HAR_vol_estimates_df

In [ ]:
"""
Obtain the target (second 10 min.) Real. Vol.

"""


## get labels of features and target

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/liquidity_features')


os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/liquidity_features')
with open('train_feat_df_reordered.pkl', 'rb') as fp:
    train_feat_df_reordered = pickle.load(fp)

df1 = train_feat_df_reordered.copy()
target_df = df1[['target','stock_id', 'time_id']]
target_df

del train_feat_df_reordered, df1



In [ ]:
"""
FULL original HAR CODE from https://github.com/jacob-hein/HAR-models-forecasting-realized-volatility-in-US-stocks/tree/main

"""


#Forecast, Errors, Betas & Plots
def HAR(data, out_sample):
    RV = data['target']

    nobs = len(RV)
    in_sample = nobs - out_sample

    outRV = RV[in_sample:]
    lag = 45 # 9 # 1 day = 9 time ids, 5 days (1 week) = 9*5 = 45 time ids
             # NOTE: lag has to be >= longest look back period. e.g. if  sum (mean) of past terms in HAR model below is 22 periods then  lag >= 22
             # in code below, lookback is 9 time ids and 45 time ids. i.e. 1 day and 5 days.

    all_predsA = np.zeros(out_sample)
    all_preds = np.zeros(out_sample)

    all_betasA = np.zeros((out_sample, 4)) # variable for parameters, all_betasA axis 1 dim. is 1 larger than in XA due to intercept term
    all_betas = np.zeros((out_sample, 4))

    ## make single step predictions each time,t
    for t in range(out_sample):
        # Estimation
        #print(f'y first index {(lag + t + 1)}' )
        #print(f'in_sample {in_sample}' )
        y = RV[lag + t + 1: in_sample + t + 1]
        XA = np.zeros((in_sample - lag, 3)) # variable for data points
        X = np.zeros((in_sample - lag, 3))

        for i in range(in_sample - lag ):
            #print(f'x first index {(-1 + i + 1 + lag + t)}' )
            # AR(3)
            XA[i, 0] = RV[-1 + i + 1 + lag + t]
            XA[i, 1] = RV[-2 + i + 1 + lag + t]
            XA[i, 2] = RV[-3 + i + 1 + lag + t]

            # HAR
            X[i, 0] = RV[-1 + i + 1 + lag + t]
            X[i, 1] = np.mean(RV[-9 + i + 1 + lag + t: i + lag + t])
            X[i, 2] = np.mean(RV[-45 + i + 1 + lag + t: i + lag + t])


        ### remove the last entry in Xs and ys during fitting to prevent lookahead bias
        XA = np.column_stack([np.ones(len(XA)), XA ]) # ones column concatenated to data for the intercept term.
        X = np.column_stack([np.ones(len(X)), X])

        # print('Full XA including prediction index = ' ,XA.shape)
        # print('Full y including prediction index = ',y.shape)

        ### remove the last entry in Xs and ys during fitting to prevent lookahead bias
        betasA = np.linalg.lstsq(XA[:-1,:], y[:-1], rcond=None)[0] # solve least squares
        betas = np.linalg.lstsq(X[:-1,:], y[:-1], rcond=None)[0]

        # print('XA[:-1,:] with prediction index excluded = ' ,XA[:-1,:].shape)
        # print('y[:-1] with prediction index excluded = ',y[:-1].shape)

        b0A, b1A, b2A, b3A = betasA
        b0, b1, b2, b3 = betas

        # print('betasA',betasA)
        # print('b0A, b1A, b2A, b3A ', b0A, b1A, b2A, b3A )

        all_betasA[t] = betasA
        all_betas[t] = betas

        # Prediction at time-step t
        #print(f"prediction time index in original data, RV[] : {in_sample + t}")

        predA = b0A + b1A * XA[-1, 1] + b2A * XA[-1, 2] + b3A * XA[-1, 3]
        pred = b0 + b1 * X[-1, 1] + b2 * X[-1, 2] + b3 * X[-1, 3]

        # Saving time-step t prediction within list for error computations
        all_predsA[t] = predA
        all_preds[t] = pred

        # this step is essentially performed at the last out of sample prediction
        # to measure model goodness of fit. It is esentially same as the least squares done abvoe.
        # Regression at t=1 for Standard Errors before performing any out-of-sample forecasts
        if t == out_sample - 1:
            modelA = LinearRegression().fit(XA[:-1,:], y[:-1])
            model = LinearRegression().fit(X[:-1,:], y[:-1])

            models_at_t_1 = {"modelA": modelA, "model": model}

            x_data = [XA[:-1,:], X[:-1,:]]
            y_data = [y[:-1], y[:-1]]

            # Below we retrieve R^2 & Adjusted R^2, prior to out-of-sample forecasts
            r_squareds = {"R-squared": [], "Adj.R-squared": []}
            for X,y, val in zip( x_data, y_data, models_at_t_1.values() ):
                r_squareds["R-squared"].append(val.score(X, y))
                r_squareds["Adj.R-squared"].append(1 - (1 - val.score(X, y)) * ((in_sample - lag - 1) / (in_sample - lag - 3)))


    # Error Computations: Mean Squared Error & Mean Absolute Error
    AR_mse = np.mean((outRV - all_predsA) ** 2)
    AR_mae = np.mean(np.abs(outRV - all_predsA))

    HAR_mse = np.mean((outRV - all_preds) ** 2)
    HAR_mae = np.mean(np.abs(outRV - all_preds))

    errors_list = {"AR_mse": AR_mse, "AR_mae": AR_mae, "HAR_mse": HAR_mse, "HAR_mae": HAR_mae}

    # Output formatting:
    ## the index of this dataframe is correct, same as out of sample prediction index in original data, RV[].
    output_df = pd.DataFrame({"stock_id": data["stock_id"][in_sample:],"time_id": data["time_id"][in_sample:], "target": outRV, "all_predsA_target": all_predsA,
                              "all_preds_target": all_preds })

    betas_list = {"all_betasA": all_betasA, "all_betas": all_betas }

    output_df_errors_betas = [output_df, errors_list, betas_list, models_at_t_1, r_squareds]

    return output_df_errors_betas



In [ ]:
"""
target HAR training model

"""

# out_sample=10 # 5
# start_time_id_index = 0
# end_time_id_index = 22*9 + out_sample # 5*9 + out_sample
unique_stock_ids = target_df['stock_id'].unique()

target_final_predictions = pd.DataFrame(index= target_df.index, columns= ["target","all_predsA_target", "all_preds_target"])
target_final_predictions[['st_id', 'time_id']] = target_df[['stock_id', 'time_id']]
target_final_predictions_errors_dict = {}


training_length = 6*9 # 5*9 (use 6 days instead of 5 bcos of error), 22*9

for s in unique_stock_ids:
    print('stock id: ',s)

    target_data = target_df[target_df['stock_id']==s]
    ind = target_data.index.values
    target_data.index = range(len(target_data))

    start_time_id_index = target_data.index[0]
    end_time_id_index = target_data.index[-1]
    data_length = end_time_id_index - start_time_id_index + 1
    out_sample = data_length - training_length

    #print(f'end_time_id_index - start_time_id_index + training_length + 1 :{ end_time_id_index - (start_time_id_index + training_length) + 1}')
    target_data = target_data.loc[start_time_id_index:end_time_id_index,:]

    target_data_output = HAR( target_data, out_sample = out_sample )

    ## save the prediction outputs.
    for col in ["target","all_predsA_target", "all_preds_target"]:
        #print(f'target_data_output[0]["{col}"].values.shape : {target_data_output[0][col].values.shape}')
        print('len',len(ind), len(target_data_output[0][col].values))
        target_final_predictions.loc[ ind[training_length:] ,col] = target_data_output[0][col].values
        target_final_predictions_errors_dict[s] = target_data_output[1]


os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
target_final_predictions.to_parquet('target_final_predictions.parquet')
pickle.dump(target_final_predictions_errors_dict,open('target_final_predictions_errors_dict.pkl','wb'))

stock id:  0
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  1
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  2
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  3
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  4
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  5
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  6
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  7
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  8
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  9
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  10
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  11
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  13
len 3829 3775
len 3829 3775
len 3829 3775
stock id:  14
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  15
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  16
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  17
len 3830 3776
len 3830 3776
len 3830 3776
stock id:  18
len 3830 3776
len 3830 3776
len 3830 3776
st

In [ ]:
"""
Evaluate book wap1 HAR training model

concatenate the intial time ids with the rest of time ids and fill missing values to crease final all HAR features

"""

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
target_final_predictions_init_time_ids =  pd.read_parquet('target_final_predictions_init_time_ids.parquet')
target_final_predictions_errors_dict_init_time_ids = pickle.load(open('target_final_predictions_errors_dict_init_time_ids.pkl','rb'))

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features')
target_final_predictions =  pd.read_parquet('target_final_predictions.parquet')
target_final_predictions_errors_dict = pickle.load(open('target_final_predictions_errors_dict.pkl','rb'))

target_all_predictions = pd.DataFrame(index= target_final_predictions.index, columns= ["target","all_predsA_target", "all_preds_target"])
HAR_types = ['target','all_predsA_target',	'all_preds_target']


for har_type in HAR_types:
    all_stock_vals = np.array([])
    for s in unique_stock_ids:
        first = target_final_predictions_init_time_ids[target_final_predictions_init_time_ids['st_id'] == s].iloc[:54][har_type].values
        second = target_final_predictions[target_final_predictions['st_id'] == s][har_type].iloc[54:].values
        last = np.concatenate((first, second))
        all_stock_vals = np.concatenate((all_stock_vals, last))
    target_all_predictions[har_type] = all_stock_vals
    target_all_predictions[har_type].ffill(inplace=True)
    target_all_predictions[har_type].bfill(inplace=True)

target_all_predictions['st_id'] = target_final_predictions['st_id']
target_all_predictions['time_id'] = target_final_predictions['time_id']

os.chdir('/content/drive/MyDrive/optiver_real_vol/Datasets/HAR_features/final_features_for_xgboost')
target_all_predictions.to_parquet('target_all_predictions.parquet')


In [ ]:
target_all_predictions

,target,all_predsA_target,all_preds_target,st_id,time_id
0,0.001453,0.000013,0.002154,0,4294
1,0.001453,0.000013,0.002154,0,31984
2,0.001453,0.000013,0.002154,0,31570
3,0.001453,0.000013,0.002154,0,5666
4,0.001453,0.000013,0.002154,0,29740
...,...,...,...,...,...
428927,0.006360,0.009503,0.006750,126,24913
428928,0.004802,0.009185,0.008249,126,15365
428929,0.003397,0.005974,0.007989,126,29316
428930,0.005479,0.004561,0.002401,126,32195
